In [1]:
import numpy as np
import tensorflow as tf

In [2]:
def map_func(inp_ids, att_mask, labels):
    return {"input_ids": inp_ids, "attention_mask": att_mask}, labels

In [3]:
with open("../input/sentiment-analysis-prep/news-ids.npy", "rb") as f:
    ids = np.load(f, allow_pickle=True)
with open("../input/sentiment-analysis-prep/news-mask.npy", "rb") as f:
    mask = np.load(f, allow_pickle=True)
with open("../input/sentiment-analysis-prep/news-labels.npy", "rb") as f:
    labels = np.load(f, allow_pickle=True)

In [4]:
ids

array([[  101,  1792,  1106, ...,     0,     0,     0],
       [  101,  7882,  2728, ...,     0,     0,     0],
       [  101,  1109,  1835, ...,     0,     0,     0],
       ...,
       [  101, 20961,  5022, ...,     0,     0,     0],
       [  101, 20820,  3813, ...,     0,     0,     0],
       [  101, 15689,  1107, ...,     0,     0,     0]])

In [5]:
mask

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])

In [6]:
labels

array([[0, 1, 0],
       [0, 1, 0],
       [1, 0, 0],
       ...,
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0]], dtype=uint8)

In [7]:
dataset = tf.data.Dataset.from_tensor_slices((ids, mask, labels))

In [8]:
dataset.take(1)

<TakeDataset shapes: ((512,), (512,), (3,)), types: (tf.int64, tf.int64, tf.uint8)>

In [9]:
dataset = dataset.map(map_func)

In [10]:
dataset.take(1)

<TakeDataset shapes: ({input_ids: (512,), attention_mask: (512,)}, (3,)), types: ({input_ids: tf.int64, attention_mask: tf.int64}, tf.uint8)>

In [11]:
batch_size = 16

In [12]:
dataset = dataset.shuffle(10000).batch(batch_size, drop_remainder=True)

In [13]:
dataset.take(1)

<TakeDataset shapes: ({input_ids: (16, 512), attention_mask: (16, 512)}, (16, 3)), types: ({input_ids: tf.int64, attention_mask: tf.int64}, tf.uint8)>

In [14]:
split = 0.8

In [15]:
train_size = int((ids.shape[0]/batch_size)*split)
print(train_size)

241


In [16]:
train_ds = dataset.take(train_size)
val_ds = dataset.skip(train_size)

In [17]:
tf.data.experimental.save(train_ds, "train")
tf.data.experimental.save(val_ds, "validation")

In [18]:
print(train_ds.element_spec == val_ds.element_spec)
print(train_ds.element_spec)
with open("element_spec.txt", "w") as f:
    f.write(str(train_ds.element_spec))

True
({'input_ids': TensorSpec(shape=(16, 512), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(16, 512), dtype=tf.int64, name=None)}, TensorSpec(shape=(16, 3), dtype=tf.uint8, name=None))
